In [1]:
# !pip install transformers
# !pip install rouge
# !pip install sentencepiece
# !pip install nomkl
# !pip install datasets
# !pip install pytorch

In [2]:
import os
import time
import numpy as np
import pandas as pd
import collections

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TFT5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import Dataset
from rouge import Rouge
import nltk.data


#### Some functions for convenience later

In [3]:
def format_story(filename):
    """Given the CNN data file, reformats to separate the story from the highlights.
    Highlights are returned as a single string"""
    
    file = open(filename,'r')
    text = file.read()
    
    # split the story and highlights
    split_text = text.split('\n\n@highlight\n\n')
    source = '\n\n@highlight\n\n'.join(split_text[:-1])
    target = split_text[-1]
    
    # return both, rejoining highlights as a single string
    return source, target

def cos_sims(out_sent, ref_sents):
    "gets cosine similarities for an output sentence with respect to the highlight sentences. Returns the sum of values."
    
    vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
    
    # get sentence level vectors with tf-idf
    tfidf = vect.fit_transform([out_sent] + ref_sents)
    
    # get similarity matrix
    similarity_mat = tfidf * tfidf.T
    
    # only values comparing "out_sent" with each sent in "ref_sents"
    return similarity_mat.toarray()[:1,1:][0]

## Load a T5 Tokenizer and Model

In [4]:
# t5 model objects
tokenizer = T5Tokenizer.from_pretrained('t5-large')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [5]:
# get list of story files in our dataset
textFiles = os.listdir('cnn_stories_tokenized')
textFiles[:5]

['0001d1afc246a7964130f43ae940af6bc6c57f01.story',
 '0002095e55fcbd3a2f366d9bf92a95433dc305ef.story',
 '00027e965c8264c35cc1bc55556db388da82b07f.story',
 '0002c17436637c4fe1837c935c04de47adb18e9a.story',
 '0003ad6ef0c37534f80b55b4235108024b407f0b.story']

In [6]:
story, highlight = format_story('cnn_stories_tokenized/'+textFiles[0])
print(story)
print('~'*50)
print(highlight)

It 's official : U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria .

Obama sent a letter to the heads of the House and Senate on Saturday night , hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons .

The proposed legislation from Obama asks Congress to approve the use of military force `` to deter , disrupt , prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction . ''

It 's a step that is set to turn an international crisis into a fierce domestic political battle .

There are key questions looming over the debate : What did U.N. weapons inspectors find in Syria ? What happens if Congress votes no ? And how will the Syrian government react ?

In a televised address from the White House Rose Garden earlier Saturday , the president said he would take his case to Congress , not because he has to 

# Let's get some baseline loss values

In [7]:
#text_files = ['example.story', 'example1.story', 'example2.story']

for i in range(5):
    
    # get formatted input and target
    story, highlights = format_story('cnn_stories_tokenized/'+textFiles[i])
    
    #train the model
    input_ids = tokenizer('summarize: ' + story, return_tensors='pt').input_ids
    labels = tokenizer(highlights, return_tensors='pt').input_ids
    
    # compute loss (this returns an array of things)
    loss = model(input_ids=input_ids, labels=labels).loss
    
    # print loss (sum of array values above)
    print(loss)

Token indices sequence length is longer than the specified maximum sequence length for this model (2412 > 512). Running this sequence through the model will result in indexing errors


tensor(4.1225, grad_fn=<NllLossBackward>)
tensor(3.1754, grad_fn=<NllLossBackward>)
tensor(3.9362, grad_fn=<NllLossBackward>)
tensor(2.8215, grad_fn=<NllLossBackward>)
tensor(3.4645, grad_fn=<NllLossBackward>)


# Train on 1000 stories (for dev purposes only. We will use a full training set for the final report)

In [22]:
source_text = []
target_text = []

start = time.time()

for i in range(2000):
    
    # get formatted input and target
    story, highlights = format_story('cnn_stories_tokenized/'+textFiles[i])
    
    source_text.append(story)
    target_text.append(highlights)
    
#     source_text.append(tokenizer('summarize: ' + story, return_tensors='tf').input_ids)
#     target_text.append(tokenizer(highlights, return_tensors='tf').input_ids)
    
    if (i+1)%50 == 0:
        print(i+1, "passed", end = ', ')
        
print('\n\ntime:', (time.time()-start)/60)

50 passed, 100 passed, 150 passed, 200 passed, 250 passed, 300 passed, 350 passed, 400 passed, 450 passed, 500 passed, 550 passed, 600 passed, 650 passed, 700 passed, 750 passed, 800 passed, 850 passed, 900 passed, 950 passed, 1000 passed, 1050 passed, 1100 passed, 1150 passed, 1200 passed, 1250 passed, 1300 passed, 1350 passed, 1400 passed, 1450 passed, 1500 passed, 1550 passed, 1600 passed, 1650 passed, 1700 passed, 1750 passed, 1800 passed, 1850 passed, 1900 passed, 1950 passed, 2000 passed, 

time: 7.072120571136475


In [24]:
train_1k = pd.DataFrame(list(zip(source_text, target_text)),columns =['source', 'target'])
train_1k = Dataset.from_pandas(train_1k)
# train_1k.iloc[0,1]

dataset = train_1k.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']
val_dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 200
})

In [25]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('t5-base')

def tokenize(batch):
    tokenized_input = tokenizer(batch['source'], padding='max_length', truncation=True)
    tokenized_label = tokenizer(batch['target'], padding='max_length', truncation=True)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=512)
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

In [26]:
val_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'source', 'target'],
    num_rows: 200
})

In [27]:
start = time.time()

output_dir = 'model-Copy1'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=10, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_name', # Wandb run name
    logging_steps=10, # How often to log loss to wandb
    eval_steps=10, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()
trainer.save_model(output_dir + '/model')

print('\n\ntime:', (time.time()-start)/60)

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
10,0.104800,0.091090,42.391900,4.718000
20,0.097500,0.089829,43.964300,4.549000
30,0.101000,0.090815,43.994800,4.546000
40,0.102000,0.090355,43.963500,4.549000
50,0.101700,0.089656,44.579200,4.486000
60,0.099500,0.089517,44.424600,4.502000
70,0.100500,0.088582,44.403500,4.504000
80,0.096800,0.088072,44.442900,4.500000
90,0.092900,0.088764,44.503500,4.494000
100,0.104200,0.087514,44.539900,4.490000




time: 48.017789463202156


# How did Training affect the loss?

In [28]:
# SECOND LOOK

our_model = T5ForConditionalGeneration.from_pretrained('model/model')

for i in range(5):
    
    # get formatted input and target
    story, highlights = format_story('cnn_stories_tokenized/'+textFiles[i])
    
    #train the model
    input_ids = tokenizer('summarize: ' + story, return_tensors='pt').input_ids
    labels = tokenizer(highlights, return_tensors='pt').input_ids
    
    # compute loss (this returns an array of things)
    loss = our_model(input_ids=input_ids, labels=labels).loss
    
    # print loss (sum of array values above)
    print(loss)

Token indices sequence length is longer than the specified maximum sequence length for this model (2412 > 512). Running this sequence through the model will result in indexing errors


tensor(2.1583, grad_fn=<NllLossBackward>)
tensor(2.1307, grad_fn=<NllLossBackward>)
tensor(2.0575, grad_fn=<NllLossBackward>)
tensor(1.7447, grad_fn=<NllLossBackward>)
tensor(2.2165, grad_fn=<NllLossBackward>)


In [14]:
# with open('story_list.txt', 'w') as f:
#     for item in textFiles:
#         f.write("%s\n" % item)

In [29]:
# format the text to input/target format
story, highlights = format_story('cnn_stories_tokenized/'+textFiles[0])

# encode the summary
encoded = tokenizer.encode('summarize: ' + story.replace('\n',' '), return_tensors='pt')

# decode
output = our_model.generate(encoded, num_beams=4, no_repeat_ngram_size=2,
                         min_length=10, max_length=400, early_stopping=True)
summary = tokenizer.decode(output[0])

print(summary)
print('='*70)
print(highlights)

<pad> NEW : U.S. has authority to act without congressional authorization, spokesman says. <unk> The aim of the game here is to ascertain whether '' chemical weapons were used in an attack in a Damascus suburb. The United States has denied that it used the weapons in the attack.</s>
Aim is to determine whether CW were used , not by whom , says U.N. spokesman


In [20]:
print(story)

-LRB- CNN -RRB- -- Juventus have sacked coach Ciro Ferrara after a string of poor results and have installed Alberto Zaccheroni in the hot seat until the end of the season .

Ferrara 's position has been threatened after a dismal run which has seen them slip out of contention in the Serie A title race as well as being eliminated from the Champions League at the group stages .

Thursday 's 2-1 Coppa Italia defeat to Italian champions Inter Milan proved the final straw and the club released a statement to confirm his departure and Zaccheroni 's arrival .

`` Zaccheroni will take over the team immediately and will guide this afternoon 's training session in Vinovo before being officially unveiled to the media , '' it read .

Ferrara joined Juve late last season and steered the Turin powerhouses to second place in Serie A behind Inter .

But his first full campaign in charge proved challenging after a promising start to the season .

The Bianconeri have lost five of their last six games in

# Okay! We have an output sentence. Let's compare it via ROUGE to the highlight that we reserved for comparing to this output at the end.

In [30]:
rouge = Rouge()
print('Output vs Story')
print('='*15)
print('ROUGE score:',rouge.get_scores(summary,highlights)[0]['rouge-1']['f'])

Output vs Story
ROUGE score: 0.23529411389705887


# There is room for improvement.